# Dataset ARM: Analisis comparativo de MLPs,CNNs y RNNs para datos basados en Time Series

In [1]:
import pandas

## Checking tensorflow backend engine
import tensorflow as tf
message = tf.constant('Hello world!')
session = tf.Session()
session.run(message)
import keras.backend as K
print(K.epsilon())

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.utils import to_categorical, plot_model

from numpy import argmax,reshape

1e-07


Using TensorFlow backend.


In [2]:
train_df = pandas.read_csv("https://raw.githubusercontent.com/e2its/datasets/master/ARM-Metric-train-TS.csv",encoding="UTF-8")

In [3]:
test_df = pandas.read_csv("https://raw.githubusercontent.com/e2its/datasets/master/ARM-Metric-test-TS.csv",encoding="UTF-8")

In [4]:
train_df.groupby('ATYPE').mean()

,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,avg_rss12_1,var_rss12_1,avg_rss13_1,var_rss13_1,avg_rss23_1,...,avg_rss12_4,var_rss12_4,avg_rss13_4,var_rss13_4,avg_rss23_4,avg_rss12_5,var_rss12_5,avg_rss13_5,var_rss13_5,avg_rss23_5
ATYPE,,,,,,,,,,,,,,,,,,,,,
bending,34.249340,0.431254,16.755905,0.733249,23.503262,34.249340,0.431254,16.755905,0.733249,23.503262,...,34.030624,0.428355,16.659172,0.727505,23.361322,33.958319,0.427648,16.626693,0.726028,23.312530
cyclin,36.743073,2.691047,16.397729,2.990645,17.391611,36.743073,2.691047,16.397729,2.990645,17.391611,...,36.514153,2.671655,16.289588,2.972231,17.278699,36.436510,2.667201,16.252897,2.965306,17.244941
lyin,39.845100,0.342937,7.264530,0.580750,9.248505,39.845100,0.342937,7.264530,0.580750,9.248505,...,39.596826,0.341302,7.222859,0.578865,9.176209,39.513958,0.340598,7.208963,0.578144,9.152296
sittin,41.743949,0.401680,14.860580,0.749794,16.095726,41.743949,0.401680,14.860580,0.749794,16.095726,...,41.489445,0.399241,14.768017,0.743606,16.006578,41.402354,0.398618,14.736741,0.742544,15.974439
standin,43.854565,0.398300,14.776667,0.615786,13.701405,43.854565,0.398300,14.776667,0.615786,13.701405,...,43.575974,0.395777,14.687328,0.611673,13.613623,43.483259,0.395032,14.655550,0.610384,13.583375
walkin,34.424189,4.337724,15.393813,3.200638,16.018848,34.424189,4.337724,15.393813,3.200638,16.018848,...,34.205685,4.311599,15.295110,3.181547,15.921445,34.137645,4.302352,15.266241,3.175534,15.886737


In [5]:
train_df.groupby('ATYPE').std()

,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,avg_rss12_1,var_rss12_1,avg_rss13_1,var_rss13_1,avg_rss23_1,...,avg_rss12_4,var_rss12_4,avg_rss13_4,var_rss13_4,avg_rss23_4,avg_rss12_5,var_rss12_5,avg_rss13_5,var_rss13_5,avg_rss23_5
ATYPE,,,,,,,,,,,,,,,,,,,,,
bending,12.471234,0.563777,7.089369,0.921850,10.430987,12.471234,0.563777,7.089369,0.921850,10.430987,...,12.725044,0.564128,7.181360,0.921409,10.562791,12.807718,0.564301,7.214285,0.921230,10.606611
cyclin,3.820435,1.934903,3.608886,1.710640,3.302717,3.820435,1.934903,3.608886,1.710640,3.302717,...,4.795252,1.939090,3.827658,1.722657,3.567922,5.076117,1.940822,3.894445,1.726250,3.652404
lyin,7.600010,0.377088,3.847469,0.668546,5.113317,7.600010,0.377088,3.847469,0.668546,5.113317,...,8.214017,0.377359,3.881109,0.669349,5.147825,8.407166,0.377567,3.892634,0.669174,5.159684
sittin,4.158324,0.421325,5.207250,0.867719,5.003480,4.158324,0.421325,5.207250,0.867719,5.003480,...,5.276706,0.421247,5.317361,0.866731,5.146924,5.604308,0.421387,5.354299,0.866980,5.193776
standin,2.628835,0.374164,3.983268,0.722081,3.907701,2.628835,0.374164,3.983268,0.722081,3.907701,...,4.350781,0.374181,4.128552,0.721632,4.046885,4.785235,0.374366,4.178717,0.721755,4.092250
walkin,4.812406,2.444056,2.916043,1.624244,3.115523,4.812406,2.444056,2.916043,1.624244,3.115523,...,5.519083,2.461158,3.151718,1.638863,3.354381,5.730645,2.466088,3.223939,1.643607,3.430036


In [6]:
test_df.groupby('ATYPE').mean()

,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,avg_rss12_1,var_rss12_1,avg_rss13_1,var_rss13_1,avg_rss23_1,...,avg_rss12_4,var_rss12_4,avg_rss13_4,var_rss13_4,avg_rss23_4,avg_rss12_5,var_rss12_5,avg_rss13_5,var_rss13_5,avg_rss23_5
ATYPE,,,,,,,,,,,,,,,,,,,,,
bending,34.597288,0.459831,17.382712,0.800169,24.515593,34.597288,0.459831,17.382712,0.800169,24.515593,...,34.597288,0.459831,17.382712,0.800169,24.515593,34.597288,0.459831,17.382712,0.800169,24.515593
cyclin,37.204545,2.430000,16.170455,2.871970,18.033939,37.204545,2.430000,16.170455,2.871970,18.033939,...,37.204545,2.430000,16.170455,2.871970,18.033939,37.204545,2.430000,16.170455,2.871970,18.033939
lyin,40.491404,0.392982,7.308246,0.604035,10.464912,40.491404,0.392982,7.308246,0.604035,10.464912,...,40.491404,0.392982,7.308246,0.604035,10.464912,40.491404,0.392982,7.308246,0.604035,10.464912
sittin,42.306897,0.370000,15.288448,0.866034,16.763276,42.306897,0.370000,15.288448,0.866034,16.763276,...,41.531034,0.370000,15.173448,0.830690,16.401207,41.531034,0.370000,15.173448,0.830690,16.401207
standin,44.255500,0.428167,14.437167,0.657500,14.179500,44.255500,0.428167,14.437167,0.657500,14.179500,...,44.255500,0.428167,14.437167,0.657500,14.179500,44.255500,0.428167,14.437167,0.657500,14.179500
walkin,35.790400,4.290000,15.501400,3.628400,16.913200,35.790400,4.290000,15.501400,3.628400,16.913200,...,35.790400,4.290000,15.501400,3.628400,16.913200,35.790400,4.290000,15.501400,3.628400,16.913200


In [7]:
test_df.groupby('ATYPE').std()

,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,avg_rss12_1,var_rss12_1,avg_rss13_1,var_rss13_1,avg_rss23_1,...,avg_rss12_4,var_rss12_4,avg_rss13_4,var_rss13_4,avg_rss23_4,avg_rss12_5,var_rss12_5,avg_rss13_5,var_rss13_5,avg_rss23_5
ATYPE,,,,,,,,,,,,,,,,,,,,,
bending,11.936775,0.423379,6.681963,0.814495,9.677460,11.936775,0.423379,6.681963,0.814495,9.677460,...,11.936775,0.423379,6.681963,0.814495,9.677460,11.936775,0.423379,6.681963,0.814495,9.677460
cyclin,3.936100,1.907624,3.280253,1.487486,3.265736,3.936100,1.907624,3.280253,1.487486,3.265736,...,3.936100,1.907624,3.280253,1.487486,3.265736,3.936100,1.907624,3.280253,1.487486,3.265736
lyin,5.823699,0.243787,3.837303,0.636870,4.170597,5.823699,0.243787,3.837303,0.636870,4.170597,...,5.823699,0.243787,3.837303,0.636870,4.170597,5.823699,0.243787,3.837303,0.636870,4.170597
sittin,3.615483,0.345055,5.476327,0.897030,5.231980,3.615483,0.345055,5.476327,0.897030,5.231980,...,6.613111,0.345055,5.724878,0.889920,5.644047,6.613111,0.345055,5.724878,0.889920,5.644047
standin,2.255689,0.629720,4.169106,0.854719,4.009013,2.255689,0.629720,4.169106,0.854719,4.009013,...,2.255689,0.629720,4.169106,0.854719,4.009013,2.255689,0.629720,4.169106,0.854719,4.009013
walkin,4.045297,2.379193,2.376477,1.605666,2.512573,4.045297,2.379193,2.376477,1.605666,2.512573,...,4.045297,2.379193,2.376477,1.605666,2.512573,4.045297,2.379193,2.376477,1.605666,2.512573


### Normalizamos los datos a mean= 0 y std = 1

In [8]:
from sklearn.preprocessing import StandardScaler as sc

In [9]:
encoder = dict()
decoder = dict()
counter = 0

for itera in train_df['ATYPE'].unique():
    encoder[itera] = counter
    decoder[counter] = itera
    counter += 1

In [10]:
train_df['ATYPE'] = train_df['ATYPE'].apply(lambda x: encoder[x])
test_df['ATYPE'] = test_df['ATYPE'].apply(lambda x: encoder[x])

In [11]:
train_df['ATYPE']

0        0
1        0
2        0
3        0
4        0
        ..
41883    5
41884    5
41885    5
41886    5
41887    5
Name: ATYPE, Length: 41888, dtype: int64

In [12]:
columns = list(train_df.columns)

In [13]:
columns.remove('ATYPE')

In [14]:
scaler = sc(copy=False)
scaler.fit(train_df[columns])

StandardScaler(copy=False, with_mean=True, with_std=True)

In [15]:
train_df[columns] = scaler.transform(train_df[columns])

In [16]:
test_df[columns] = scaler.transform(test_df[columns])

In [17]:
train_df.groupby('ATYPE').mean()

,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,avg_rss12_1,var_rss12_1,avg_rss13_1,var_rss13_1,avg_rss23_1,...,avg_rss12_4,var_rss12_4,avg_rss13_4,var_rss13_4,avg_rss23_4,avg_rss12_5,var_rss12_5,avg_rss13_5,var_rss13_5,avg_rss23_5
ATYPE,,,,,,,,,,,,,,,,,,,,,
0,-0.575895,-0.500222,0.459846,-0.463988,1.106898,-0.575895,-0.500222,0.459846,-0.463988,1.106898,...,-0.532057,-0.497881,0.450926,-0.461891,1.086105,-0.519429,-0.497130,0.447944,-0.461080,1.079195
1,-0.243751,0.601693,0.395822,0.910000,0.226360,-0.243751,0.601693,0.395822,0.910000,0.226360,...,-0.224814,0.597723,0.385914,0.904619,0.221410,-0.219566,0.597236,0.382540,0.902363,0.220416
2,0.169412,-0.543286,-1.236753,-0.556808,-0.946862,0.169412,-0.543286,-1.236753,-0.556808,-0.946862,...,0.156550,-0.540397,-1.208972,-0.552378,-0.930425,0.152806,-0.539667,-1.199911,-0.551124,-0.924980
3,0.422323,-0.514642,0.121054,-0.453917,0.039654,0.422323,-0.514642,0.121054,-0.453917,0.039654,...,0.390691,-0.512100,0.118262,-0.452089,0.040568,0.381303,-0.511316,0.117253,-0.451024,0.040595
4,0.703439,-0.516291,0.106054,-0.535483,-0.305308,0.703439,-0.516291,0.106054,-0.535483,-0.305308,...,0.648820,-0.513792,0.104068,-0.532405,-0.299610,0.633094,-0.513068,0.103046,-0.531493,-0.297825
5,-0.552607,1.404641,0.216370,1.037814,0.028578,-0.552607,1.404641,0.216370,1.037814,0.028578,...,-0.510400,1.398655,0.210980,1.032044,0.028466,-0.497730,1.396258,0.209901,1.030366,0.028182


In [18]:
test_df.groupby('ATYPE').mean()

,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,avg_rss12_1,var_rss12_1,avg_rss13_1,var_rss13_1,avg_rss23_1,...,avg_rss12_4,var_rss12_4,avg_rss13_4,var_rss13_4,avg_rss23_4,avg_rss12_5,var_rss12_5,avg_rss13_5,var_rss13_5,avg_rss23_5
ATYPE,,,,,,,,,,,,,,,,,,,,,
0,-0.529551,-0.486287,0.571889,-0.423256,1.252751,-0.529551,-0.486287,0.571889,-0.423256,1.252751,...,-0.461954,-0.482509,0.578201,-0.417655,1.250194,-0.442113,-0.481404,0.580228,-0.415937,1.249471
1,-0.182287,0.474402,0.355196,0.837767,0.318903,-0.182287,0.474402,0.355196,0.837767,0.318903,...,-0.139404,0.479702,0.364958,0.843584,0.328774,-0.126634,0.481326,0.368115,0.845533,0.332087
2,0.255495,-0.518883,-1.228939,-0.542635,-0.771608,0.255495,-0.518883,-1.228939,-0.542635,-0.771608,...,0.267221,-0.515156,-1.193952,-0.537055,-0.747225,0.271078,-0.514069,-1.182539,-0.535359,-0.739198
3,0.497303,-0.530090,0.197536,-0.383166,0.135832,0.497303,-0.530090,0.197536,-0.383166,0.135832,...,0.395836,-0.526381,0.189579,-0.399076,0.096668,0.396874,-0.525300,0.193665,-0.397354,0.100998
4,0.756840,-0.501727,0.045368,-0.510093,-0.236427,0.756840,-0.501727,0.045368,-0.510093,-0.236427,...,0.732886,-0.497973,0.060064,-0.504508,-0.219166,0.726535,-0.496877,0.064835,-0.502805,-0.213452
5,-0.370639,1.381370,0.235601,1.298176,0.157432,-0.370639,1.381370,0.235601,1.298176,0.157432,...,-0.314352,1.388107,0.247268,1.304072,0.169452,-0.297746,1.390222,0.251048,1.306105,0.173463


In [19]:
x_train = train_df[columns].to_numpy()
y_train = to_categorical(train_df['ATYPE'].to_numpy())

In [20]:
x_test = test_df[columns].to_numpy()
y_test = to_categorical(test_df['ATYPE'].to_numpy())

### Establecemos las estructuras de almacenamiento e iteración de nuestro modelo

In [21]:
batch_size = [32, 256]
epochs = [25]
perceptrons = [64, 256, 1024]
dropout = [0.05]

In [22]:
model_result = {'conf': None, 'xaccuracy': None, 'yaccuracy': None, 'xloss' : None, 'yloss' : None, 
                'bepoch' : None, 'baccuracy': None, 'bloss': None}
models_analysis = list()

### Importamos las clases Keras necesarias para la creacion de nuestra MLP básica y componemos el modelo.

In [23]:
input_dimension = len(train_df.columns)-1
output_dimension = len(train_df['ATYPE'].unique())

In [24]:
for hl1 in perceptrons:
    for hl2 in perceptrons:
        for bs in batch_size:
            for ep in epochs:
                for dp in dropout:
                    model = Sequential()
                    model.add(Dense(hl1, input_shape=(input_dimension,)))
                    model.add(Activation('relu'))
                    model.add(Dropout(dp))
                    model.add(Dense(hl2))
                    model.add(Activation('relu'))
                    model.add(Dropout(dp))
                    model.add(Dense(output_dimension))
                    model.add(Activation('softmax'))

                    mr = model_result.copy()
                    mr['conf'] = [hl1, hl2, bs, ep, dp]            

                    # loss function for one-hot vector
                    # use of adam optimizer
                    # accuracy is a good metric for classification tasks
                    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

                    # train the network
                    model_history = model.fit(x=x_train, y=y_train, batch_size=bs, epochs=ep)
                    index_acc = argmax(model_history.history['acc'])
                    mr['bloss'] = model_history.history['loss'][-1]
                    mr['baccuracy'] = model_history.history['acc'][-1]
                    mr['bepoch']= index_acc + 1
                    mr['xloss'] = model_history.history['loss'][index_acc]
                    mr['xaccuracy'] = model_history.history['acc'][index_acc]

                    # Evaluate the network                                                             
                    mr['yloss'], mr['yaccuracy'] = model.evaluate(x_test,y_test, batch_size=bs)
                    models_analysis.append(mr)   





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/25
41888/41888 [==============================] - 13s 302us/step - loss: 0.7599 - acc: 0.67810s - loss: 0.764
Epoch 2/25
41888/41888 [==============================] - 9s 218us/step - loss: 0.6563 - acc: 0.7104
Epoch 3/25
41888/41888 [==============================] - 6s 132us/step - loss: 0.6304 - acc: 0.7217
Epoch 4/25
41888/41888 [==============================] - 10s 232us/step - loss: 0.6152 - acc: 0.7284
Epoch 5/25
41888/41888 [==============================] - 12s 297us/step - loss: 0.6027 - acc: 0.7351
Epoch 6/25
41888/41888 [==============================] - 7s 175us/step - loss: 0.5923 - acc: 0.7393
Epoch 7/25
41888/41888 [==============================] - 6s 145us/step - loss: 0.5845 - acc: 0.7431
Epoch 8/25
41888/41888 [==============================] - 6

41888/41888 [==============================] - 9s 224us/step - loss: 0.5388 - acc: 0.7634
Epoch 12/25
41888/41888 [==============================] - 10s 233us/step - loss: 0.5354 - acc: 0.7653
Epoch 13/25
41888/41888 [==============================] - 10s 243us/step - loss: 0.5304 - acc: 0.7686
Epoch 14/25
41888/41888 [==============================] - 7s 169us/step - loss: 0.5294 - acc: 0.7686
Epoch 15/25
41888/41888 [==============================] - 8s 198us/step - loss: 0.5278 - acc: 0.7694
Epoch 16/25
41888/41888 [==============================] - 8s 183us/step - loss: 0.5226 - acc: 0.7718
Epoch 17/25
41888/41888 [==============================] - 8s 185us/step - loss: 0.5206 - acc: 0.7734
Epoch 18/25
41888/41888 [==============================] - 11s 273us/step - loss: 0.5188 - acc: 0.7725
Epoch 19/25
41888/41888 [==============================] - 8s 202us/step - loss: 0.5172 - acc: 0.7729
Epoch 20/25
41888/41888 [==============================] - 5s 130us/step - loss: 0.5141 - a

41888/41888 [==============================] - 2s 59us/step - loss: 0.5380 - acc: 0.7625: 0s - loss: 0.5377 - acc: 0.762
Epoch 15/25
41888/41888 [==============================] - 2s 55us/step - loss: 0.5339 - acc: 0.7647
Epoch 16/25
41888/41888 [==============================] - 2s 59us/step - loss: 0.5309 - acc: 0.7653
Epoch 17/25
41888/41888 [==============================] - 3s 73us/step - loss: 0.5312 - acc: 0.7640
Epoch 18/25
41888/41888 [==============================] - 2s 51us/step - loss: 0.5267 - acc: 0.7712
Epoch 19/25
41888/41888 [==============================] - 2s 50us/step - loss: 0.5239 - acc: 0.7706
Epoch 20/25
41888/41888 [==============================] - 3s 60us/step - loss: 0.5209 - acc: 0.7692
Epoch 21/25
41888/41888 [==============================] - 3s 62us/step - loss: 0.5188 - acc: 0.7728
Epoch 22/25
41888/41888 [==============================] - 3s 62us/step - loss: 0.5169 - acc: 0.7745
Epoch 23/25
41888/41888 [==============================] - 3s 78us/step

41888/41888 [==============================] - 11s 263us/step - loss: 0.5042 - acc: 0.7804
Epoch 18/25
41888/41888 [==============================] - 8s 203us/step - loss: 0.5026 - acc: 0.7821
Epoch 19/25
41888/41888 [==============================] - 17s 411us/step - loss: 0.4989 - acc: 0.7808
Epoch 20/25
41888/41888 [==============================] - 10s 248us/step - loss: 0.4957 - acc: 0.7822
Epoch 21/25
41888/41888 [==============================] - 8s 190us/step - loss: 0.4959 - acc: 0.7829
Epoch 22/25
41888/41888 [==============================] - 9s 215us/step - loss: 0.4938 - acc: 0.7850
Epoch 23/25
41888/41888 [==============================] - 12s 275us/step - loss: 0.4915 - acc: 0.7853
Epoch 24/25
41888/41888 [==============================] - 14s 333us/step - loss: 0.4873 - acc: 0.7869
Epoch 25/25
350/350 [==============================] - 0s 1ms/step
Epoch 1/25
41888/41888 [==============================] - 4s 95us/step - loss: 0.7879 - acc: 0.6655
Epoch 2/25
41888/41888 [

41888/41888 [==============================] - 4s 101us/step - loss: 0.5055 - acc: 0.7781
Epoch 19/25
41888/41888 [==============================] - 3s 76us/step - loss: 0.5019 - acc: 0.7812
Epoch 20/25
41888/41888 [==============================] - 3s 76us/step - loss: 0.4995 - acc: 0.7808
Epoch 21/25
41888/41888 [==============================] - 3s 71us/step - loss: 0.4976 - acc: 0.7809
Epoch 22/25
41888/41888 [==============================] - 3s 72us/step - loss: 0.4938 - acc: 0.7840
Epoch 23/25
41888/41888 [==============================] - 3s 73us/step - loss: 0.4923 - acc: 0.7842
Epoch 24/25
41888/41888 [==============================] - 3s 70us/step - loss: 0.4902 - acc: 0.7859
Epoch 25/25
350/350 [==============================] - 0s 946us/step
Epoch 1/25
41888/41888 [==============================] - 12s 285us/step - loss: 0.7151 - acc: 0.6901
Epoch 2/25
41888/41888 [==============================] - 17s 415us/step - loss: 0.6295 - acc: 0.7213
Epoch 3/25
41888/41888 [=======

41888/41888 [==============================] - 17s 405us/step - loss: 0.4946 - acc: 0.7853
Epoch 21/25
41888/41888 [==============================] - 14s 337us/step - loss: 0.4938 - acc: 0.78321s - l
Epoch 22/25
41888/41888 [==============================] - 22s 526us/step - loss: 0.4903 - acc: 0.7878
Epoch 23/25
41888/41888 [==============================] - 16s 371us/step - loss: 0.4887 - acc: 0.7871
Epoch 24/25
41888/41888 [==============================] - 16s 388us/step - loss: 0.4871 - acc: 0.7895
Epoch 25/25
350/350 [==============================] - 0s 1ms/step
Epoch 1/25
41888/41888 [==============================] - 5s 109us/step - loss: 0.7454 - acc: 0.6812 1s - loss: 0.771
Epoch 2/25
41888/41888 [==============================] - 3s 79us/step - loss: 0.6403 - acc: 0.7177
Epoch 3/25
41888/41888 [==============================] - 5s 111us/step - loss: 0.6055 - acc: 0.7334 2s - loss: 0.6130 - acc: 0 - ETA: 2s
Epoch 4/25
41888/41888 [==============================] - 4s 99us/st

41888/41888 [==============================] - 7s 157us/step - loss: 0.4925 - acc: 0.7865
Epoch 20/25
41888/41888 [==============================] - 6s 150us/step - loss: 0.4880 - acc: 0.7877
Epoch 21/25
41888/41888 [==============================] - 6s 150us/step - loss: 0.4875 - acc: 0.7875
Epoch 22/25
41888/41888 [==============================] - 8s 199us/step - loss: 0.4838 - acc: 0.7883
Epoch 23/25
41888/41888 [==============================] - 10s 234us/step - loss: 0.4801 - acc: 0.7910
Epoch 24/25
41888/41888 [==============================] - 7s 157us/step - loss: 0.4805 - acc: 0.7898
Epoch 25/25
350/350 [==============================] - 0s 1ms/step


### MLP best network

In [25]:
best_model = None

for itera in models_analysis:
    if best_model is None:
        best_model = itera
    elif itera['baccuracy'] > best_model['baccuracy']:
        best_model = itera

print (best_model)

{'conf': [1024, 1024, 32, 25, 0.05], 'xaccuracy': 0.7920406799083269, 'yaccuracy': 0.8028571442195347, 'xloss': 0.4790971497026812, 'yloss': 0.4716323331424168, 'bepoch': 25, 'baccuracy': 0.7920406799083269, 'bloss': 0.4790971497026812}


### Testing code MLP (sequential APi)

In [30]:
model = Sequential()
model.add(Dense(best_model['conf'][0], input_shape=(input_dimension,)))
model.add(Activation('relu'))
model.add(Dropout(best_model['conf'][4]))
model.add(Dense(best_model['conf'][1]))
model.add(Activation('relu'))
model.add(Dropout(best_model['conf'][4]))
model.add(Dense(output_dimension))
model.add(Activation('softmax'))

# loss function for one-hot vector
# use of adam optimizer
# accuracy is a good metric for classification tasks
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# train the network
model_history = model.fit(x=x_train, y=y_train, batch_size=best_model['conf'][2], epochs=best_model['bepoch'])

a,b = model.evaluate(x_test,y_test, batch_size=best_model['conf'][2])

Epoch 1/25
41888/41888 [==============================] - 35s 831us/step - loss: 0.6963 - acc: 0.6939
Epoch 2/25
41888/41888 [==============================] - 21s 507us/step - loss: 0.6005 - acc: 0.7350
Epoch 3/25
41888/41888 [==============================] - 27s 642us/step - loss: 0.5747 - acc: 0.7466
Epoch 4/25
41888/41888 [==============================] - 21s 511us/step - loss: 0.5595 - acc: 0.7513
Epoch 5/25
41888/41888 [==============================] - 26s 615us/step - loss: 0.5485 - acc: 0.7567
Epoch 6/25
41888/41888 [==============================] - 31s 734us/step - loss: 0.5407 - acc: 0.7614
Epoch 7/25
41888/41888 [==============================] - 23s 557us/step - loss: 0.5305 - acc: 0.7670
Epoch 8/25
41888/41888 [==============================] - 32s 766us/step - loss: 0.5267 - acc: 0.7683
Epoch 9/25
41888/41888 [==============================] - 25s 597us/step - loss: 0.5203 - acc: 0.77371s - loss: 0.
Epoch 10/25
41888/41888 [==============================] - 32s 767us/

In [ ]:
plot_model(model, to_file='./img/mlp-best-model-sAPI.png', show_shapes=True)

In [31]:
print ('Train : Loss(%s) - Accuracy: (%s)' %(model_history.history['loss'][-1],model_history.history['acc'][-1]))

Train : Loss(0.47486138588558907) - Accuracy: (0.7921600458365164)


In [32]:
print ('Test : Loss(%s) - Accuracy: (%s)' %(a,b))

Test : Loss(0.4890320369175502) - Accuracy: (0.7942857156481061)


### Testing code MLP (Functional API)

In [39]:
from keras.models import Model
from keras.layers import Dense, Activation, Dropout, Input

input_1 = Input(shape=(input_dimension,))
x = input_1
x = Dense(best_model['conf'][0])(x)
x = Activation('relu')(x)
x = Dropout(best_model['conf'][4])(x)
x = Dense(best_model['conf'][1])(x)
x = Activation('relu')(x)
x = Dropout(best_model['conf'][4])(x)
output = Dense(output_dimension, activation = 'softmax' )(x)
model = Model([input_1], output)

# loss function for one-hot vector
# use of adam optimizer
# accuracy is a good metric for classification tasks
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# train the network
model_history = model.fit(x=x_train, y=y_train, batch_size=best_model['conf'][2], epochs=best_model['bepoch'])

a,b = model.evaluate(x_test,y_test, batch_size=best_model['conf'][2])

Epoch 1/25
41888/41888 [==============================] - 35s 840us/step - loss: 0.6929 - acc: 0.6935
Epoch 2/25
41888/41888 [==============================] - 30s 713us/step - loss: 0.6026 - acc: 0.7338
Epoch 3/25
41888/41888 [==============================] - 32s 775us/step - loss: 0.5738 - acc: 0.7478
Epoch 4/25
41888/41888 [==============================] - 23s 558us/step - loss: 0.5613 - acc: 0.7523
Epoch 5/25
41888/41888 [==============================] - 33s 795us/step - loss: 0.5489 - acc: 0.7600
Epoch 6/25
41888/41888 [==============================] - 25s 598us/step - loss: 0.5382 - acc: 0.7637
Epoch 7/25
41888/41888 [==============================] - 29s 687us/step - loss: 0.5327 - acc: 0.7658
Epoch 8/25
41888/41888 [==============================] - 23s 551us/step - loss: 0.5282 - acc: 0.76820s - loss: 0.5281 - acc: 0.768
Epoch 9/25
41888/41888 [==============================] - 29s 699us/step - loss: 0.5206 - acc: 0.7720
Epoch 10/25
41888/41888 [===========================

In [ ]:
plot_model(model, to_file='./img/mlp-best-model-fAPI.png', show_shapes=True)

In [40]:
print ('Train : Loss(%s) - Accuracy: (%s)' %(model_history.history['loss'][-1],model_history.history['acc'][-1]))

Train : Loss(0.47873473711556563) - Accuracy: (0.7926136363636364)


In [41]:
print ('Test : Loss(%s) - Accuracy: (%s)' %(a,b))

Test : Loss(0.46824740512030466) - Accuracy: (0.8199999986376081)
